##### Authors: Alexander Mo & Tommaso Lucarelli

# Feature pipeline for Yahoo Finance data
This pipeline takes a set of stock market ticker symbols and retrieves historical data and statistics from Yahoo Finance. The data is then processed to fit the training pipeline's LSTM network. Upon processing the data, the features will be stored using Google Drive where they can be retrieved on demand.

In [ ]:
!pip install yfinance --upgrade --no-cache-dir
!pip install pandas

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [1]:
import yfinance as yf
import pandas as pd
from random import randint

In [1]:
from stock_processing_functions import processStock

,Open,High,Low,Close,Volume,Dividends,Stock Splits,Diff,EPS Estimate,Reported EPS,Offset
0,149.500000,149.880005,148.785004,149.449997,10322310,0.0,0.0,-0.050003,1.27,1.29,0.0157
1,149.460007,149.969193,148.970001,149.289993,4350752,0.0,0.0,-0.170013,1.27,1.29,0.0157
2,149.289993,149.369995,147.940002,148.839996,4434843,0.0,0.0,-0.449997,1.27,1.29,0.0157
3,148.830002,149.070007,147.994995,148.089996,4137871,0.0,0.0,-0.740005,1.27,1.29,0.0157
4,148.080093,148.229996,146.910004,147.449997,3214674,0.0,0.0,-0.630096,1.27,1.29,0.0157
...,...,...,...,...,...,...,...,...,...,...,...
235,130.039993,130.039993,129.179993,129.328903,4455911,0.0,0.0,-0.711090,1.27,1.29,0.0157
236,129.339905,129.339996,128.720001,129.239899,4720025,0.0,0.0,-0.100006,1.27,1.29,0.0157
237,129.240005,130.279999,128.979996,130.165207,3862308,0.0,0.0,0.925201,1.27,1.29,0.0157
238,130.160004,130.294998,129.710007,130.020004,2872984,0.0,0.0,-0.139999,1.27,1.29,0.0157


In [ ]:
# Not needed with yfinance on Python 3.9

def parseMonth(month):
    """
    Function to parse the string of a month from the earnings date
    :param month: string with the given month
    :return: the number corresponding to the month in datetime format
    """
    if month == 'Jan':
        return 1
    elif month == 'Feb':
        return 2
    elif month == 'Mar':
        return 3
    elif month == 'Apr':
        return 4
    elif month == 'May':
        return 5
    elif month == 'Jun':
        return 6
    elif month == 'Jul':
        return 7
    elif month == 'Aug':
        return 8
    elif month == 'Sep':
        return 9
    elif month == 'Oct':
        return 10
    elif month == 'Nov':
        return 11
    else:
        return 12

# Not needed with yfinance on Python 3.9
def convertEarningsDate(df):
    """
    Function to convert the earnings date format to datetime
    :param df: dataframe containing the earnings history
    :return: dataframe updated with the parsed datetime as date
    """
    test = [None]*len(df)
    for i in range(len(df)):
        temp = df.loc[i, "Earnings Date"].split(', ')
        temp = [x.strip() for x in temp]
        temp = (temp[0] + ' ' + temp[1]).split(' ')
        test[i] = [temp[2], parseMonth(temp[0]), temp[1]]
    dt = pd.DataFrame(test, columns=['year', 'month', 'day'])
    df['Earnings Date'] = pd.to_datetime(dt)
    return df

### Functions to process the data of a stock into the right format
Final dataframe contains all necessary columns for the model in the right format and dtype.

In [ ]:
def getEarnings(stock):
    """
    Function to get earnings and process the date indexing
    :param stock: yf.Ticker object of the stock to process
    :return: earnings of the respective stock, processed.
    """
    earn = stock.get_earnings_dates()
    if earn is not None:
        earn['Earnings Date'] = earn.index.date
        earn['Earnings Date'] = pd.to_datetime(earn['Earnings Date'])
    return earn

def getHistory(symbol, stock, period='1mo', interval='15m'):
    """
    Function to retrieve the price history of the stock and parse its date
    :param stock: yfinance ticker object
    :param period: the period over which data should be collected
    :param interval: the interval for data points
    :return: history dataframe with additional columns
    """
    hist = stock.history(period = period, interval = interval)
    hist['company'] = symbol
    hist['date'] = hist.index.date
    hist['date'] = pd.to_datetime(hist['date'])
    hist['Diff'] = hist['Close'] - hist['Open']
    return hist

def getRelEarnings(e_df, hist_df):
    """
    Finds the earnings data which is relevant for the given history time frame
    :param e_df: earnings dataframe
    :param hist_df: history dataframe
    :return: relevant dates dataframe
    """
    e_df.reset_index(inplace=True, drop=True)
    minmax = hist_df['date'].agg(['min', 'max'])
    last_er_idx = e_df[e_df['Earnings Date'] <= minmax['min']].index[0]
    first_er_idx = e_df[e_df['Earnings Date'] <= minmax['max']].index[0]
    relevant_earnings = e_df[first_er_idx:last_er_idx+1].reset_index(drop=True)
    return relevant_earnings

def fillEarnings(current, hist_df, idx_in):
    """
    Function to fill the earnings columns into the history
    :param current: dataframe holding the earnings data for the selected indices
    :param hist_df: history dataframe
    :param idx_in: relevant indices on the history dataframe to fill the earnings data for
    :return: history dataframe with the earnings added for the given indices
    """
    hist_df.loc[idx_in, 'EPS Estimate'] = current['EPS Estimate']
    hist_df.loc[idx_in, 'Reported EPS'] = current['Reported EPS']
    hist_df.loc[idx_in, 'Offset'] = current['Surprise(%)']
    hist_df.loc[idx_in, 'Earnings'] = current['Earnings Date']
    return hist_df


def getHistWithEarnings(relevant_earnings, hist_df):
    """
    Function to add the corresponding earnings data to the days for which the data was known.
    :param relevant_earnings: The earnings columns which are relevant for the given history time frame
    :param hist_df: The history dataframe
    :return: History with added columns for each of the relevant earnings
    """
    for idx in reversed(relevant_earnings.index):
        if idx>0:
            current = relevant_earnings.iloc[idx]
            next = relevant_earnings.iloc[idx-1]
            idx_in = hist_df[(hist_df['date'] >= current['Earnings Date']) &
                             (hist_df['date'] < next['Earnings Date'])].index
            hist_df = fillEarnings(current, hist_df, idx_in)
        else:
            current = relevant_earnings.iloc[idx]
            idx_in = hist_df[(hist_df['date'] >= current['Earnings Date'])].index
            hist_df = fillEarnings(current, hist_df, idx_in)
    return hist_df

def dropIrrelevant(hist_df: pd.DataFrame):
    """
    Drop the columns which are not needed for the model
    :param hist_df: dataframe containing the history with all other attribute columns.
    :return: same df with the columns dropped.
    """
    labels = ['date', 'Earnings', 'company']
    return hist_df.drop(labels, axis=1)

### Functions for selecting stocks and formatting

In [ ]:
def stockToCSV(symbol, period='1mo', interval='15m'):
    """
    Functions to write stock information to CSV
    :param symbol: ticker symbol of the company
    :param period: over what time period the history data should be taken
    :param interval: how often a sample is taken over the period
    :return:
    """
    stock = yf.Ticker(symbol)
    earnings = getEarnings(stock)
    # earnings = convertEarningsDate(earnings)
    hist = getHistory(symbol, stock, period=period, interval=interval)
    rel_earnings = getRelEarnings(earnings, hist)
    hist = getHistWithEarnings(rel_earnings, hist)
    hist = dropIrrelevant(hist)
    hist.to_csv('./data/' + symbol)

def stockToDf(symbol, earnings, period='1mo', interval='15m'):
    """
    Functions to write stock information to CSV
    :param symbol: ticker symbol of the company
    :param earnings: dataframe containing the respective earnings
    :param period: over what time period the history data should be taken
    :param interval: how often a sample is taken over the period
    :return:
    """
    stock = yf.Ticker(symbol)
    # earnings = getEarnings(stock)
    hist = getHistory(symbol, stock, period=period, interval=interval)
    rel_earnings = getRelEarnings(earnings, hist)
    hist = getHistWithEarnings(rel_earnings, hist)
    hist = dropIrrelevant(hist)
    return hist

def getStocks():
    """
    Get all stock symbols listed on Nasdaq
    :return: list with symbols
    """
    df = pd.read_table('http://www.nasdaqtrader.com/dynamic/symdir/nasdaqlisted.txt')
    # local run
    # df = pd.read_table('tickers.txt')
    symbols = [None]*len(df)
    for idx, line in df.iterrows():
        symbols[idx] = line[0].split('|')[0]
    return symbols

def selectStocks(stocks, n):
    """
    Select n random stocks from the list
    :param stocks: list of company symbols
    :param n: number of companies to select
    :return: selected symbols
    """
    idxs = []
    selected = [None]*n
    earnings = [None]*n
    counter = 0
    while len(idxs) < n:
        temp_int = randint(0, len(stocks))
        if len(stocks) == n:
            temp_int = counter
        if temp_int not in idxs:
            earn = getEarnings(yf.Ticker(stocks[temp_int]))
            if earn is not None:
                idxs.append(temp_int)
                selected[counter] = stocks[temp_int]
                earnings[counter] = earn
                counter += 1
    return selected, earnings

def selectAll(stocks):
    """
    Select all stocks which have enough data on yfinance listed on the Nasdaq list
    :param stocks: list of company symbols fom Nasdaq
    :return: current symbols
    """
    selected = []
    earnings = []
    for stock in stocks:
        try:
            temp = getEarnings(yf.Ticker(stock))
        except:
            print('error loading the stock')
        if temp is not None:
            selected.append(stock)
            earnings.append(temp)
    return selected, earnings


### Run and store features on Google Drive

In [ ]:
from google.colab import drive
drive.mount('/gdrive')

%cd /gdrive/MyDrive/Scalable/Project/feature_store

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).
/gdrive/MyDrive/Scalable/Project/feature_store


In [ ]:
# PARAMETERS FOR RUN
SET = False
N = 5
ONE_CSV = True
PATH = './features.csv'

def runAll(n):
    stocks = getStocks()
    if SET:
        selected, earnings = selectStocks(stocks, n)
    else:
        stocks = ['AAPL', 'TSLA', 'AMZN']
        selected, earnings = selectStocks(stocks, len(stocks))
    if ONE_CSV:
        df = pd.DataFrame()
        for idx, stock in enumerate(selected):
            df = df.append(stockToDf(selected[idx], earnings[idx]))
        df.reset_index(drop=True, inplace=True)
        with open(PATH, 'w', encoding = 'utf-8-sig') as f:
            df.to_csv(f)
    else:
        for stock in selected:
            stockToCSV(stock)


In [ ]:
runAll(N)

['AACG', 'AACI', 'AACIU', 'AACIW', 'AADI', 'AADR', 'AAL', 'AAME', 'AAOI', 'AAON', 'AAPB', 'AAPD', 'AAPL', 'AAPU', 'AAWW', 'AAXJ', 'ABCB', 'ABCL', 'ABCM', 'ABEO', 'ABGI', 'ABIO', 'ABNB', 'ABOS', 'ABSI', 'ABST', 'ABUS', 'ABVC', 'ACAB', 'ACABU', 'ACABW', 'ACAC', 'ACACU', 'ACACW', 'ACAD', 'ACAH', 'ACAHU', 'ACAHW', 'ACAX', 'ACAXR', 'ACAXU', 'ACAXW', 'ACB', 'ACBA', 'ACBAU', 'ACBAW', 'ACCD', 'ACDC', 'ACDCW', 'ACER', 'ACET', 'ACGL', 'ACGLN', 'ACGLO', 'ACHC', 'ACHL', 'ACHV', 'ACIU', 'ACIW', 'ACLS', 'ACLX', 'ACMR', 'ACNB', 'ACNT', 'ACON', 'ACONW', 'ACOR', 'ACQR', 'ACQRU', 'ACQRW', 'ACRS', 'ACRV', 'ACRX', 'ACST', 'ACT', 'ACTG', 'ACVA', 'ACWI', 'ACWX', 'ACXP', 'ADAG', 'ADAL', 'ADALU', 'ADALW', 'ADAP', 'ADBE', 'ADD', 'ADEA', 'ADER', 'ADERU', 'ADERW', 'ADES', 'ADI', 'ADIL', 'ADILW', 'ADMA', 'ADMP', 'ADN', 'ADNWW', 'ADOC', 'ADOCR', 'ADOCW', 'ADP', 'ADPT', 'ADRE', 'ADSE', 'ADSEW', 'ADSK', 'ADTH', 'ADTHW', 'ADTN', 'ADTX', 'ADUS', 'ADV', 'ADVM', 'ADVWW', 'ADXN', 'AEAC', 'AEACU', 'AEACW', 'AEAE', 'AEAEU'